## Build Functions for ELT

In [7]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [8]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Download, Transform, and Modeling All in One

In [9]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [16]:
f.predictions('tsla')

TSLA 5m Interval Timestamp: 2024-11-11 12:21:37 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,up,up
kelly_1:2.5,0.184674,0.136398,0.135914,0.097447
prob_up,0.415911,0.388426,0.399586,0.375
prob_static,0.154524,0.219185,0.232689,0.25
prob_down,0.429565,0.392389,0.367725,0.375
precision,0.417625,0.383142,0.382796,0.355319
recall,0.426614,0.391389,0.347656,0.326172
f1,0.422072,0.387222,0.364381,0.340122
support,"[512.0, 509.0, 511.0]","[512.0, 509.0, 511.0]","[512.0, 509.0, 511.0]","[512.0, 509.0, 511.0]"


TSLA 15m Interval Timestamp: 2024-11-11 12:21:44 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.17095,0.103659,0.132609,0.092121
prob_up,0.408024,0.539748,0.437759,0.375
prob_static,0.248102,0.235147,0.285657,0.25
prob_down,0.343874,0.225105,0.276584,0.375
precision,0.407821,0.359756,0.380435,0.351515
recall,0.431953,0.349112,0.414201,0.343195
f1,0.41954,0.354354,0.396601,0.347305
support,"[169.0, 169.0, 168.0]","[169.0, 169.0, 168.0]","[169.0, 169.0, 168.0]","[169.0, 169.0, 168.0]"


TSLA 1h Interval Timestamp: 2024-11-11 12:21:48 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,up,up
kelly_1:2.5,0.101167,0.052766,0.111828,0.120354
prob_up,0.348147,0.304242,0.398455,0.5
prob_static,0.287734,0.322586,0.24788,0.125
prob_down,0.36412,0.373172,0.353665,0.375
precision,0.357977,0.323404,0.365591,0.371681
recall,0.386555,0.319328,0.284519,0.351464
f1,0.371717,0.321353,0.32,0.36129
support,"[239.0, 240.0, 238.0]","[239.0, 240.0, 238.0]","[239.0, 240.0, 238.0]","[239.0, 240.0, 238.0]"


TSLA 1wk Interval Timestamp: 2024-11-11 12:22:01 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.075472,0.066667,0.092593,0.109091
prob_up,0.690613,0.49678,0.486559,0.375
prob_static,0.117235,0.157696,0.257253,0.25
prob_down,0.192152,0.345525,0.256188,0.375
precision,0.339623,0.333333,0.351852,0.363636
recall,0.375,0.333333,0.395833,0.333333
f1,0.356436,0.333333,0.372549,0.347826
support,"[48.0, 48.0, 48.0]","[48.0, 48.0, 48.0]","[48.0, 48.0, 48.0]","[48.0, 48.0, 48.0]"


TSLA 1mo Interval Timestamp: 2024-11-11 12:22:03 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,up
kelly_1:2.5,-0.088889,-0.088889,-0.26,0.246154
prob_up,0.134435,0.198677,0.253608,0.375
prob_static,0.057048,0.242539,0.192948,0.25
prob_down,0.808517,0.558784,0.553444,0.375
precision,0.222222,0.222222,0.1,0.461538
recall,0.2,0.2,0.1,0.666667
f1,0.210526,0.210526,0.1,0.545455
support,"[9.0, 9.0, 10.0]","[9.0, 9.0, 10.0]","[9.0, 9.0, 10.0]","[9.0, 9.0, 10.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')